In [ ]:
import dautil as dl
import ch7util
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
context = dl.nb.Context('portfolio_optimization')
lr = dl.nb.LatexRenderer(chapter=7, start=16, context=context)
lr.render(r'''\operatorname{E}(R_p) =  w_A \operatorname{E}(R_A) +
w_B \operatorname{E}(R_B) = w_A \operatorname{E}(R_A) + (1 - w_A) \operatorname{E}(R_B)''')
lr.render(r'\sigma_p^2  = w_A^2 \sigma_A^2  + w_B^2 \sigma_B^2 + 2w_Aw_B  \sigma_{A} \sigma_{B} \rho_{AB}')

In [ ]:
def expected_return(stocka, stockb, means):
    return 0.5 * (means[stocka] + means[stockb])

In [ ]:
def variance_return(stocka, stockb, stds):
    ohlc = dl.data.OHLC()
    dfa = ohlc.get(stocka)
    dfb = ohlc.get(stockb)
    merged = pd.merge(left=dfa, right=dfb,
                      right_index=True, left_index=True,
                      suffixes=('_A', '_B')).dropna()
    retsa = ch7util.log_rets(merged['Adj Close_A'])
    retsb = ch7util.log_rets(merged['Adj Close_B'])
    corr = np.corrcoef(retsa, retsb)[0][1]

    return 0.25 * (stds[stocka] ** 2 + stds[stockb] ** 2 +
                   2 * stds[stocka] * stds[stockb] * corr)

In [ ]:
def calc_ratio(stocka, stockb, means, stds, ratios):
    if stocka == stockb:
        return np.nan

    key = stocka + '_' + stockb
    ratio = ratios.get(key, None)

    if ratio:
        return ratio

    expected = expected_return(stocka, stockb, means)
    var = variance_return(stocka, stockb, stds)
    ratio = expected/var
    ratios[key] = ratio

    return ratio

In [ ]:
means = {}
stds = {}

ohlc = dl.data.OHLC()

for stock in ch7util.STOCKS:
    close = ohlc.get(stock)['Adj Close']
    rets = ch7util.log_rets(close)
    means[stock] = rets.mean()
    stds[stock] = rets.std()

In [ ]:
pairs = dl.collect.grid_list(ch7util.STOCKS)
sorted_pairs = [[sorted(row[i]) for row in pairs]
                for i in range(len(ch7util.STOCKS))]
ratios = {}

grid = [[calc_ratio(row[i][0], row[i][1], means, stds, ratios)
        for row in sorted_pairs] for i in range(len(ch7util.STOCKS))]

In [ ]:
%matplotlib inline
plt.title('Expected Return/Return Variance for 2 Asset Portfolio')
sns.heatmap(grid, xticklabels=ch7util.STOCKS, yticklabels=ch7util.STOCKS)